In [45]:
from functions.data_by_country import data_by_country
from utils.data_loader import load_data_parquet
import matplotlib.pyplot as plt
import pandas as pd
import pycountry
import plotly.graph_objects as go

In [21]:
df = load_data_parquet()
df.dtypes

dt                               datetime64[ns]
AverageTemperature                      float64
AverageTemperatureUncertainty           float64
City                                   category
Country                                category
Latitude                                 object
Longitude                                object
dtype: object

In [22]:
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [23]:
df = data_by_country(df)
df = df.dropna()

#df = df.pivot(index="dt", columns="Country",
#              values='AverageTemperature').dropna()

# %%
# data = df.loc["2013-01-01"] - df.loc["1889-01-01"]
# data = data.sort_values()

c:\Users\gaute\OneDrive\Malmø\1st semester\AI\World_Temperature_Analysis\functions\data_by_country.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data = data[["Country", "AverageTemperature", "dt"]


In [24]:
# Function to get ISO codes for countries
def get_iso_code(country_name):
    try:
        country = pycountry.countries.get(name=country_name)
        return country.alpha_3
    except AttributeError:
        return None

df['Code'] = df['Country'].apply(get_iso_code)

print(df)








            Country         dt  AverageTemperature Code
1070    Afghanistan 1833-01-01            1.145000  AFG
1071    Afghanistan 1833-02-01            2.168400  AFG
1072    Afghanistan 1833-03-01            6.516200  AFG
1073    Afghanistan 1833-04-01           12.832200  AFG
1074    Afghanistan 1833-05-01           18.544800  AFG
...             ...        ...                 ...  ...
514995     Zimbabwe 2013-04-01           19.770143  ZWE
514996     Zimbabwe 2013-05-01           17.851714  ZWE
514997     Zimbabwe 2013-06-01           16.527143  ZWE
514998     Zimbabwe 2013-07-01           15.858857  ZWE
514999     Zimbabwe 2013-08-01           18.746571  ZWE

[374572 rows x 4 columns]


In [25]:
df.head()

,Country,dt,AverageTemperature,Code
1070,Afghanistan,1833-01-01,1.1450,AFG
1071,Afghanistan,1833-02-01,2.1684,AFG
1072,Afghanistan,1833-03-01,6.5162,AFG
1073,Afghanistan,1833-04-01,12.8322,AFG
1074,Afghanistan,1833-05-01,18.5448,AFG


In [26]:
# Data per year

df2 = df.groupby(df['dt'].dt.year)

In [27]:
df2

In [38]:
new_df = []

for i in df2:
    print(i)
    new_df.append(i)
    
# new_df = pd.DataFrame(
#     new_df
# )

(1743,                        Country         dt  AverageTemperature  Code
3239                   Albania 1743-11-01           12.686000   ALB
22673                  Austria 1743-11-01            2.607800   AUT
38868                  Belarus 1743-11-01            1.128875   BLR
42107                  Belgium 1743-11-01            7.363571   BEL
51824   Bosnia And Herzegovina 1743-11-01            6.615800   BIH
61541                 Bulgaria 1743-11-01            6.456714   BGR
80975                   Canada 1743-11-01           -0.206687   CAN
110126                 Croatia 1743-11-01            6.622333   HRV
119843          Czech Republic 1743-11-01            4.530200  None
126321                 Denmark 1743-11-01            6.221000   DNK
152233                 Estonia 1743-11-01            0.269500   EST
158711                 Finland 1743-11-01            0.619000   FIN
161950                  France 1743-11-01            7.299972   FRA
174906                 Germany 1743-11-01

In [44]:
df = df[df["dt"]=="2013"]

In [31]:
filtered_year

,Country,dt,AverageTemperature,Code
3230,Afghanistan,2013-01-01,1.874000,AFG
6469,Albania,2013-01-01,9.055000,ALB
9708,Algeria,2013-01-01,10.116200,DZA
12947,Angola,2013-01-01,24.526000,AGO
16186,Argentina,2013-01-01,24.425688,ARG
...,...,...,...,...
502036,Venezuela,2013-01-01,25.176175,None
505275,Vietnam,2013-01-01,21.801733,None
508514,Yemen,2013-01-01,23.152500,YEM
511753,Zambia,2013-01-01,22.695750,ZMB


In [58]:

# Different colors https://plotly.com/python/builtin-colorscales/



fig = go.Figure(data=go.Choropleth(
    locations = df['Code'],
    z = df['AverageTemperature'],
    text = df['Country'],
    colorscale = 'Hot',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = 'Celsius',
    colorbar_title = 'Average Temperatures',
    
))

# fig.update_layout(
#     title_text='2014 Global GDP',
#     geo=dict(
#         showframe=False,
#         showcoastlines=False,
#         projection_type='equirectangular'
#     ),
#     annotations = [dict(
#         x=0.55,
#         y=0.1,
#         xref='paper',
#         yref='paper',
#         text='Source: <a href="https://www.cia.gov/library/publications/the-world-factbook/fields/2195.html">\
#             CIA World Factbook</a>',
#         showarrow = False
#     )]
# )

fig.show()